In [1]:

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import json

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras import backend as K

batch_size = 128
nb_classes = 10
nb_epoch = 80

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3
# number of convolutional filters to use
nb_filters = 5
# convolution kernel size
nb_conv = 5
# size of pooling area for max pooling
nb_pool = 2

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()


Using TensorFlow backend.


In [2]:
if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], img_channels, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], img_channels, img_rows, img_cols)
    input_shape = (img_channels, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, img_channels)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, img_channels)
    input_shape = (img_rows, img_cols, img_channels)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [4]:
model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

C:\Users\Kimsarah\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (5, 5), input_shape=(32, 32, 3..., padding="valid")`
C:\Users\Kimsarah\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, (5, 5))`


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 5)         380       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 5)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 5)         630       
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 5)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 5)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 5)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 720)               0         
__________

In [6]:
hist = model.fit(X_train, Y_train, batch_size=batch_size, 
                 nb_epoch=nb_epoch, verbose=1, validation_split=0.2)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

C:\Users\Kimsarah\Anaconda3\lib\site-packages\keras\models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 40000 samples, validate on 10000 samples
Epoch 1/80
40000/40000 [==============================] - 58s - loss: 2.0624 - acc: 0.2373 - val_loss: 1.8978 - val_acc: 0.3196
Epoch 2/80
40000/40000 [==============================] - 58s - loss: 1.8151 - acc: 0.3394 - val_loss: 1.6589 - val_acc: 0.4055
Epoch 3/80
40000/40000 [==============================] - 57s - loss: 1.7031 - acc: 0.3806 - val_loss: 1.5948 - val_acc: 0.4271
Epoch 4/80
40000/40000 [==============================] - 57s - loss: 1.6372 - acc: 0.4049 - val_loss: 1.5469 - val_acc: 0.4445
Epoch 5/80
40000/40000 [==============================] - 58s - loss: 1.5956 - acc: 0.4202 - val_loss: 1.5064 - val_acc: 0.4594
Epoch 6/80
40000/40000 [==============================] - 58s - loss: 1.5672 - acc: 0.4294 - val_loss: 1.5029 - val_acc: 0.4601
Epoch 7/80
40000/40000 [==============================] - 58s - loss: 1.5391 - acc: 0.4423 - val_loss: 1.4581 - val_acc: 0.4838
Epoch 8/80
40000/40000 [==============================

In [7]:
import os
os.chdir('C:/Users/Kimsarah/Desktop/SK_ML/code')

# Save the model and weights
json_string = model.to_json()
open('cifar10_model_architecture.json','w').write(json_string)
model.save_weights('cifar10_model_weights.h5')

# Save History
with open('cifar10_model_history.json','w') as fp:
    json.dump(hist.history, fp)